In [3]:
#Custom code in this library
from cleaner_utils import super_cleaner
from pretraining_data_utils import make_book_token_frequency, token_freq_df_to_dict, \
                                    all_available_tokens_from_df, optimize_book_subset, optimize_book_subset_ratio
from pretraining_data_utils import book_properties, make_df_book_properties
from gutenberg.acquire import load_etext


from tokenizer.tokenizer import StrategizedTokenizer
from dataset.dataset import StrategizedTokenizerDataset
from dataset.dataset import DefaultTokenizerDataset

#Training code
from transformers import BertConfig
from transformers import BertForMaskedLM
from transformers import BertTokenizer
from transformers import AdamW
from transformers import Trainer, TrainingArguments

from torch.utils.data import DataLoader

#General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import re
import json
import torch
import logging
import pickle
import os

from more_itertools import take
from datetime import datetime

In [4]:
cache_dir = 'cached_files/'

In [5]:
# Read scraped metadata from the gutenberg metadata database 
#(Original data was scraped by using https://github.com/c-w/gutenberg)
#The data is then further preprocessed by https://github.com/hugovk/gutenberg-metadata so it is actually usable.

f = open(cache_dir + 'gutenberg-metadata.json', 'r')
metadata = json.load(f)
f.close()

In [6]:
#retrieve how many english books there in english
english_book_keys = [key for key in metadata.keys() if metadata[key]['language'] == ['en']]
len(english_book_keys)

13142

In [7]:
# The third book cant be retrieved because of faults in retrieval. This happens sometimes.
import traceback
import sys

try:
    super_cleaner(load_etext(14575), -1, verify_deletions=True)
except Exception as e:
    try:
        exc_info = sys.exc_info()
    finally:
        # Display the *original* exception
        traceback.print_exception(*exc_info)
        del exc_info


Traceback (most recent call last):
  File "<ipython-input-7-0358ce9648a3>", line 6, in <module>
    super_cleaner(load_etext(14575), -1, verify_deletions=True)
  File "C:\Users\s145733\Anaconda3\lib\site-packages\gutenberg\acquire\text.py", line 152, in load_etext
    text = cache.read().decode('utf-8')
  File "C:\Users\s145733\Anaconda3\lib\gzip.py", line 292, in read
    return self._buffer.read(size)
  File "C:\Users\s145733\Anaconda3\lib\gzip.py", line 470, in read
    self._read_eof()
  File "C:\Users\s145733\Anaconda3\lib\gzip.py", line 516, in _read_eof
    raise BadGzipFile("CRC check failed %s != %s" % (hex(crc32),
gzip.BadGzipFile: CRC check failed 0x0 != 0xd0c5998f


In [8]:
#retrieve how many english books there are actually loadable
#If books arent cached this may take a while because it needs to scrape the books from gutenberg.org
#Therefore i provide a pre-processed file
if os.path.isfile(cache_dir + 'loadable_english_book_keys.pkl'):
    with open(cache_dir + 'loadable_english_book_keys.pkl', 'rb') as f:
        loadable_english_book_keys = pickle.load(f)              
else:
    loadable_english_book_keys = []
    i = 0
    for key in english_book_keys:
        if i % 1000 == 0:
            print(i, datetime.now())
        i += 1
        try:
            load_etext(int(key))
            loadable_english_book_keys.append(key)
        except:
            continue
    with open(cache_dir + 'loadable_english_book_keys.pkl', 'wb') as f:
        pickle.dump(loadable_english_book_keys, f)
            
len(loadable_english_book_keys)

12640

In [9]:
#Randomly select 20 books that we can query
np.random.seed(42)
rand_10_books = [x for x in np.random.choice(loadable_english_book_keys, size=10)]
print(rand_10_books)

['17255', '1742', '14870', '14596', '23436', '22563', '15306', '15976', '1344', '13579']


In [10]:
# Titles and authors for the first 5 books
# 1 book isnt actually loadable, see below.
for book_id in rand_10_books[:5]:
    print(book_id, metadata[book_id]['author'], metadata[book_id]['title'])

17255 ['Alma-Tadema, Laurence'] ['The Wings of Icarus: Being the Life of one Emilia Fletcher']
1742 ['Davis, Richard Harding'] ['Miss Civilization: A Comedy in One Act']
14870 ['Hopkinson, Alfred, Sir'] ['Rebuilding Britain: A Survey of Problems of Reconstruction After the World War']
14596 ['Inge, William Ralph'] ['Christian Mysticism']
23436 ['Anonymous'] ['Aladdin or The Wonderful Lamp']


Text preprocessing
===================

In [11]:
# original unprocessed text
text = load_etext(50000)[:500]
text

'The Project Gutenberg EBook of John Gutenberg, by Franz von Dingelstedt\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org/license\r\n\r\n\r\nTitle: John Gutenberg\r\n       First Master Printer, His Acts and Most Remarkable\r\n       Discourses and his Death\r\n\r\nAuthor: Franz von Dingelstedt\r\n\r\nRelease Da'

In [12]:
#Text with formatting
print(text)

The Project Gutenberg EBook of John Gutenberg, by Franz von Dingelstedt

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: John Gutenberg
       First Master Printer, His Acts and Most Remarkable
       Discourses and his Death

Author: Franz von Dingelstedt

Release Da


Use the cleaner to retrieve cleaned text from the first book of the random selection.
The _super_cleaner_ strips a headers/disclaimers/tables that are not required for our purposes.

In [13]:

sentences = super_cleaner(load_etext(16968), -1, verify_deletions=False)

In [14]:
#Text is now a list of paragraphs
sentences[:10]

[' "And now for business," Lopez said. "And remember zat he what tells a lie shall be right away shotted." In his excitement he lost the little English he had.',
 ' "Put all ze men outside," Lopez ordered. Venustiano and Pedro, his chief lieutenants, obeyed at once, forcing them to march ahead of them, and standing guard over them near a great cactus bush a few feet from the adobe. "Leave ze women with me," the bandit continued. "But first, Alvarada, you find ze cook. I am \'ongry."',
 ' "Red" Giddings had been on the ranch with Gilbert since the very beginning. He came from the North with the young man, willing to stake all on this one venture. Like young Jones, he was not afraid. He was an efficient, well-set-up young fellow, with three consuming passions: Arizona, his harmonica, and Angela Hardy. The first saw a lot of "Red"; the second touched his lips frequently; but as for Angela--well, perhaps the poor boy kissed his harmonica so often in order to forget her lips. But if his own

In [15]:
#with some short sentences
sorted(sentences, key=len)[:20]

['"No."',
 '"Gun?"',
 '"Why?"',
 '"Yes."',
 '"Pells?"',
 '"A what?"',
 '"I have?"',
 '"Joking?"',
 '"Really?"',
 '"I ain\'t!"',
 '"Kiss me!"',
 '"Uh--huh!"',
 '"In a way."',
 '"What for?"',
 '"Yes, sir!"',
 '"Yes; why?"',
 'She nodded.',
 '"All those?"',
 '"You won\'t?"',
 'She started.']

In [16]:
list(sentences)

[' "And now for business," Lopez said. "And remember zat he what tells a lie shall be right away shotted." In his excitement he lost the little English he had.',
 ' "Put all ze men outside," Lopez ordered. Venustiano and Pedro, his chief lieutenants, obeyed at once, forcing them to march ahead of them, and standing guard over them near a great cactus bush a few feet from the adobe. "Leave ze women with me," the bandit continued. "But first, Alvarada, you find ze cook. I am \'ongry."',
 ' "Red" Giddings had been on the ranch with Gilbert since the very beginning. He came from the North with the young man, willing to stake all on this one venture. Like young Jones, he was not afraid. He was an efficient, well-set-up young fellow, with three consuming passions: Arizona, his harmonica, and Angela Hardy. The first saw a lot of "Red"; the second touched his lips frequently; but as for Angela--well, perhaps the poor boy kissed his harmonica so often in order to forget her lips. But if his own

In [17]:
#Find some properties about the book
book_properties(sentences)

[2048, 5, 1532, 75745, 353]

Tokenization in practice
============

In [18]:
#initialize custom tokenizer
ST_tokenizer = StrategizedTokenizer(padding=False)

In [19]:
inputs = ST_tokenizer.tokenize("Anne went to the Albert Heijn at 5 o'clock to buy some milk for me.")
inputs

{'input_ids': tensor([[  101,   103,  2253,  2000,  1996,   103,   103,   103,   103,  2012,
          1019,  1051,  1005,  5119,  2000,  4965,  2070,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,   103,  2000,  1996,  4789,  2002, 28418,  2078,  2012,
          1019,  1051,  1005,  5119,  2000,   103,  2070,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,  2253,   103,  1996,  4789,  2002, 28418,  2078,   103,
          1019,  1051,  1005,  5119,   103,  4965,  2070,  6501,   103,  2033,
          1012,   102],
        [  101,  4776,  2253,  2000,   103,  4789,  2002, 28418,  2078,  2012,
          1019,  1051,  1005,  5119,  2000,  4965,   103,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,  2253,  2000,  1996,  4789,  2002, 28418,  2078,  2012,
           103,  1051,  1005,  5119,  2000,  4965,  2070,  6501,  2005,  2033,
          1012,   102],
        [  101,  4776,  2253,  2000,  1996,  4789,  2002, 28418,  2078,  201

In [20]:
#Masks are at different places
for masked_line in ST_tokenizer.convert_ids_to_tokens(inputs['input_ids']):
    print(masked_line)

['[CLS]', '[MASK]', 'went', 'to', 'the', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'at', '5', 'o', "'", 'clock', 'to', 'buy', 'some', 'milk', 'for', 'me', '.', '[SEP]']
['[CLS]', 'anne', '[MASK]', 'to', 'the', 'albert', 'he', '##ij', '##n', 'at', '5', 'o', "'", 'clock', 'to', '[MASK]', 'some', 'milk', 'for', 'me', '.', '[SEP]']
['[CLS]', 'anne', 'went', '[MASK]', 'the', 'albert', 'he', '##ij', '##n', '[MASK]', '5', 'o', "'", 'clock', '[MASK]', 'buy', 'some', 'milk', '[MASK]', 'me', '.', '[SEP]']
['[CLS]', 'anne', 'went', 'to', '[MASK]', 'albert', 'he', '##ij', '##n', 'at', '5', 'o', "'", 'clock', 'to', 'buy', '[MASK]', 'milk', 'for', 'me', '.', '[SEP]']
['[CLS]', 'anne', 'went', 'to', 'the', 'albert', 'he', '##ij', '##n', 'at', '[MASK]', 'o', "'", 'clock', 'to', 'buy', 'some', 'milk', 'for', 'me', '.', '[SEP]']
['[CLS]', 'anne', 'went', 'to', 'the', 'albert', 'he', '##ij', '##n', 'at', '5', 'o', "'", 'clock', 'to', 'buy', 'some', '[MASK]', 'for', 'me', '.', '[SEP]']
['[CLS]', 'anne', 'we

Gutenberg book-selection
==============

In [21]:
#Setting to ignore warnings about sequences being longer than BERT can handle
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)
from datetime import datetime
print(datetime.now())
df_books_10 = make_df_book_properties(rand_10_books)
print(datetime.now())

2021-04-30 14:24:20.005476
2021-04-30 14:24:36.260769


In [22]:
df_books_10

,book_id,num_sentences,Shortest sentence (char),Longest sentence (char),Total tokens,Longest sequence (tokens)
0,17255,677,8,2083,41672,522
1,1742,240,15,1051,8534,253
2,14870,319,22,5189,63786,1022
3,14596,541,18,7685,122124,1887
4,23436,11,41,293,579,71
5,22563,364,6,2344,16734,530
6,15306,390,9,1672,40223,383
7,15976,1453,10,1724,81203,453
8,1344,263,6,6391,27092,1536
9,13579,872,13,1939,65723,447


In [23]:
#Sort df and account for the fact that the column has both text and numbers
df_books_10.sort_values(by='Shortest sentence (char)')[:10]

,book_id,num_sentences,Shortest sentence (char),Longest sentence (char),Total tokens,Longest sequence (tokens)
5,22563,364,6,2344,16734,530
8,1344,263,6,6391,27092,1536
0,17255,677,8,2083,41672,522
6,15306,390,9,1672,40223,383
7,15976,1453,10,1724,81203,453
9,13579,872,13,1939,65723,447
1,1742,240,15,1051,8534,253
3,14596,541,18,7685,122124,1887
2,14870,319,22,5189,63786,1022
4,23436,11,41,293,579,71


In [24]:
#Some books have very few tokens.
df_books_10.sort_values(by='Total tokens').head()

,book_id,num_sentences,Shortest sentence (char),Longest sentence (char),Total tokens,Longest sequence (tokens)
4,23436,11,41,293,579,71
1,1742,240,15,1051,8534,253
5,22563,364,6,2344,16734,530
8,1344,263,6,6391,27092,1536
6,15306,390,9,1672,40223,383


In [25]:
df_books_10.sort_values(by='Total tokens').tail()

,book_id,num_sentences,Shortest sentence (char),Longest sentence (char),Total tokens,Longest sequence (tokens)
0,17255,677,8,2083,41672,522
2,14870,319,22,5189,63786,1022
9,13579,872,13,1939,65723,447
7,15976,1453,10,1724,81203,453
3,14596,541,18,7685,122124,1887


In [26]:
#Retrieve token occurences per book in a dataframe and another dataframe with total number of tokens
print(datetime.now())
df_book_token_freq_10, df_10_total_tokens = make_book_token_frequency(rand_10_books)
print(datetime.now())

2021-04-30 14:24:36.348601


100%|██████████| 10/10 [00:09<00:00,  1.05it/s]

2021-04-30 14:24:47.678579


In [27]:
# DataFrame is obviously very sparse
df_book_token_freq_10[:10]

,[PAD],[unused0],[unused1],[unused2],[unused3],[unused4],[unused5],[unused6],[unused7],[unused8],...,##！,##（,##）,##，,##－,##．,##／,##：,##？,##～
17255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1742,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14870,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15306,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1344,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#total number of tokens per book
df_10_total_tokens

17255     41681.0
1742       8546.0
14870     63788.0
14596    122126.0
23436       581.0
22563     16736.0
15306     40225.0
15976     81249.0
1344      27094.0
13579     65725.0
dtype: float64

In [29]:
#Total number of tokens in our small set
df_10_total_tokens.sum()

467751.0

In [30]:
#All tokens which are present in our subsample of 20 books
all_present_tokens_10 = all_available_tokens_from_df(df_book_token_freq_10)
all_present_tokens_10, len(all_present_tokens_10)

(array([  101,   102,   999, ..., 29645, 29664, 29667], dtype=int64), 15268)

In [31]:
#Show first 3 entries
tokens_per_book_10 = token_freq_df_to_dict(df_book_token_freq_10, df_10_total_tokens)
take(3, tokens_per_book_10.items())

[('17255',
  {'tokens': array([  101,   102,   999, ..., 29591, 29602, 29667], dtype=int64),
   'total_tokens': 41681.0}),
 ('1742',
  {'tokens': array([  101,   102,   999, ..., 28838, 29122, 29586], dtype=int64),
   'total_tokens': 8546.0}),
 ('14870',
  {'tokens': array([  101,   102,   999, ..., 29598, 29602, 29609], dtype=int64),
   'total_tokens': 63788.0})]

In [32]:
optimize_book_subset(all_present_tokens_10, tokens_per_book_10, threshold = 1e5)

book best:  14870 new tokens:  6042
book best:  22563 new tokens:  1639
book best:  1742 new tokens:  345
book best:  23436 new tokens:  67


{'subset_booklist': ['14870', '22563', '1742', '23436'],
 'subset_total_tokens': 89651.0,
 'subset_present_tokens': array([  101.,   102.,   999., ..., 29602., 29609., 29664.]),
 'subset_unique_tokens': 8093}

In [33]:
print(optimize_book_subset_ratio(all_present_tokens_10, tokens_per_book_10, threshold = 1e5))

book best:  22563 new tokens:  3683 book_total_tokens:  16736.0 ratio:  0.22006453154875716
book best:  23436 new tokens:  123 book_total_tokens:  581.0 ratio:  0.2117039586919105
book best:  1344 new tokens:  2198 book_total_tokens:  27094.0 ratio:  0.08112497231859453
book best:  1742 new tokens:  367 book_total_tokens:  8546.0 ratio:  0.04294406739995319
book best:  15306 new tokens:  2153 book_total_tokens:  40225.0 ratio:  0.053523927905531385
{'subset_booklist': ['22563', '23436', '1344', '1742', '15306'], 'subset_total_tokens': 93182.0, 'subset_present_tokens': array([  101.,   102.,   999., ..., 29602., 29664., 29667.]), 'subset_unique_tokens': 8524}


In [30]:
df_book_token_freq = pd.read_csv(cache_dir + 'df_book_token_freq.csv', index_col=0)
df_book_token_freq.head()

,[PAD],[unused0],[unused1],[unused2],[unused3],[unused4],[unused5],[unused6],[unused7],[unused8],...,##！,##（,##）,##，,##－,##．,##／,##：,##？,##～
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1881,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1882,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1883,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df_total_tokens = pd.read_csv(cache_dir +'df_total_tokens.csv', index_col=0).squeeze()
df_total_tokens.head()

1         7642.0
10      901309.0
11       36117.0
12       40833.0
13        6778.0
          ...   
1880    227995.0
1881     98655.0
1882     65838.0
1883    100735.0
1884     15394.0
Name: 0, Length: 1000, dtype: float64

In [33]:
df_total_tokens.sort_values()

Int64Index([ 239,  117,  156,  232,  127,  129,  212,  114,  116, 1525, 1016,
             115, 1546, 1593, 1065, 1544,  104, 1545, 1341,  235],
           dtype='int64')

In [47]:
if os.path.isfile(cache_dir + 'subset_meta_100K.pkl'):
    with open(cache_dir + 'subset_meta_100K.pkl', 'rb') as f:
        subset_100K = pickle.load(f)
        
for k,v in subset_100K.items():
    print(k,v)

subset_booklist ['1061', '1034', '114']
subset_total_tokens 99978.0
subset_present_tokens [  101.   102.   999. ... 29604. 29606. 29611.]
subset_unique_tokens 9320


In [48]:
if os.path.isfile(cache_dir + 'subset_meta_ratio_100K.pkl'):
    with open(cache_dir + 'subset_meta_ratio_100K.pkl', 'rb') as f:
        subset_ratio_100K = pickle.load(f)
for k,v in subset_ratio_100K.items():
    print(k,v)

subset_booklist ['117', '127', '1525', '116', '1546', '1321', '232', '104', '1567', '235', '1016', '1137', '1359', '1064', '1593', '1757', '1336', '1475', '1861', '115', '1331', '1171', '212', '114']
subset_total_tokens 99181.0
subset_present_tokens [  101.   102.   999. ... 29731. 29735. 29739.]
subset_unique_tokens 10892


In [49]:
if os.path.isfile(cache_dir + 'subset_meta_1M.pkl'):
    with open(cache_dir + 'subset_meta_1M.pkl', 'rb') as f:
        subset_1M = pickle.load(f)
for k,v in subset_1M.items():
    print(k,v)

subset_booklist ['22', '1662', '1558', '1034', '235']
subset_total_tokens 999936.0
subset_present_tokens [  101.   102.   999. ... 29604. 29607. 29667.]
subset_unique_tokens 20841


In [50]:
if os.path.isfile(cache_dir + 'subset_meta_ratio_1M.pkl'):
    with open(cache_dir + 'subset_meta_ratio_1M.pkl', 'rb') as f:
        subset_ratio_1M = pickle.load(f)
for k,v in subset_ratio_1M.items():
    print(k,v)

subset_booklist ['117', '127', '1525', '116', '1546', '1321', '232', '104', '1567', '235', '1016', '1137', '1359', '1064', '1593', '1757', '1336', '1475', '1861', '115', '1331', '1042', '180', '1034', '1870', '1092', '1462', '1663', '1222', '1219', '1031', '101', '212', '1566', '112', '1746', '1221', '1817', '1855', '162', '114', '1179', '1063', '1805', '1615', '1645', '1280', '118', '1215', '1753', '1080', '1189']
subset_total_tokens 999805.0
subset_present_tokens [  100.   101.   102. ... 29736. 29737. 29739.]
subset_unique_tokens 21996


In [51]:
if os.path.isfile(cache_dir + 'subset_meta_10M.pkl'):
    with open(cache_dir + 'subset_meta_10M.pkl', 'rb') as f:
        subset_10M = pickle.load(f)
for k,v in subset_10M.items():
    print(k,v)

subset_booklist ['200', '22', '1662', '101', '1449', '162', '135', '14', '1365', '1156', '118', '1340', '1444', '100', '1615', '1226', '1725', '1391', '1351', '1694', '1399', '1162', '1320', '1452', '152', '1166', '1218', '1198', '1479', '112', '1304', '180', '1804', '1039', '1210', '1470', '1558', '1060', '235', '116', '114', '115', '1065', '1593']
subset_total_tokens 9997943.0
subset_present_tokens [  100.   101.   102. ... 29736. 29737. 29739.]
subset_unique_tokens 26396


In [52]:
if os.path.isfile(cache_dir + 'subset_meta_ratio_10M.pkl'):
    with open(cache_dir + 'subset_meta_ratio_10M.pkl', 'rb') as f:
        subset_ratio_10M = pickle.load(f)
for k,v in subset_ratio_10M.items():
    print(k,v)

subset_booklist ['117', '127', '1525', '116', '1546', '1321', '232', '104', '1567', '235', '1016', '1137', '1359', '1064', '1593', '1757', '1336', '1475', '1861', '115', '1331', '1042', '180', '1034', '1870', '1092', '1462', '1663', '1222', '1219', '1031', '101', '212', '1566', '112', '1746', '1221', '1817', '1855', '162', '114', '1179', '1063', '1805', '1615', '1280', '118', '1324', '1753', '1708', '1317', '1425', '1645', '1479', '1444', '14', '1418', '1351', '1551', '1873', '1270', '1864', '22', '1818', '1199', '1060', '1446', '1166', '152', '1367', '1210', '1065', '1804', '1638', '1149', '1742', '1847', '1451', '1491', '1234', '1187', '1477', '1662', '1808', '1220', '1758', '1054', '1278', '1764', '1253', '1233', '1478', '1694', '1395', '1470', '1761', '1763', '200', '1670', '1186', '132', '1240', '1816', '1669', '1830', '1020', '1387', '240', '1273', '1050', '1206', '1820', '1045', '1156', '1195', '1594', '1391', '1859', '1824', '1827', '1734', '1169', '1731', '1554', '1160', '1165

In [66]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
tokenizer.vocab

In [ ]:
for k,v in tokenizer.vocab.items():
    if not v in subset_10M['subset_present_tokens'] and not k.startswith('[unused'):
        print(k, v)


In [37]:
from tokenizer.tokenizer import StrategizedTokenizer

In [38]:
tok = StrategizedTokenizer()
tok.__str__()

'<tokenizer.tokenizer.StrategizedTokenizer object at 0x000001980E674880>'

In [69]:
tokenizer.__str__()

'<transformers.tokenization_bert.BertTokenizer object at 0x000001FD97F5F070>'

In [42]:
[type(x) for x in super_cleaner(load_etext(1), -1)]

[numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,
 numpy.str_,

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
text = "Anne went to the Albert Heijn at 5 o'clock to buy some milk for me."

In [ ]:
default_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
default_tokenizer(text)

In [ ]:
ST_tokenizer = StrategizedTokenizer(padding=True)
inputs = ST_tokenizer.tokenize(text)
inputs

In [ ]:
for x in ST_tokenizer.convert_ids_to_tokens(inputs['input_ids']):
    print(x)

In [ ]:
train_dataset = MODataset()
train_dataset.populate()

In [ ]:
train_dataset.labels

In [ ]:
bert_tiny_config = {"hidden_size": 128, 
                    "hidden_act": "gelu", 
                    "initializer_range": 0.02, 
                    "vocab_size": 30522, 
                    "hidden_dropout_prob": 0.1, 
                    "num_attention_heads": 2, 
                    "type_vocab_size": 2, 
                    "max_position_embeddings": 512, 
                    "num_hidden_layers": 2, 
                    "intermediate_size": 512, 
                    "attention_probs_dropout_prob": 0.1}

model = BertForMaskedLM(config=BertConfig(**bert_tiny_config))
model.train();

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    #per_device_eval_batch_size=256,   # batch size for evaluation
    learning_rate=1e-5,     
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=None            # evaluation dataset
)

In [ ]:
#trainer.train()